In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import yfinance as yf

### Introduction

### Hurst Exponent 

The Hurst exponent was originally introduced by Harold Edwin Hurst as an indicator of long-range
memory.

The fBm is a generalization of the standard Bm. Increments of the fBm are Gaussian, since the  
fBm is a fractional integral or a fractional derivative of a Bm, Wt

$$ B_H(t) = \frac{\sigma}{\Gamma(H+\frac{1}{2})}\int((t-s)^{H - \frac{1}{2}}_+ - (-s)^{H - \frac{1}{2}}_+)dW(s)$$

The two parameters of the fBm are the Hurst exponent H, and the volatility parameter.  
If H = 1/2, the fBm is a Bm. If H > 1/2 (respectively H < 1/2), the fBm is the fractional integral  
(resp. fractional derivative) of order H-1/2 (resp. 1/2-H) of a Bm; increments are thus positively  
(resp. negatively) correlated.

### Hurst exponent estimation

The absolute-moment method uses another definition of the fBm, which is consistent with the  
integral form provided above. Indeed, an fBm is also the only zero-mean Gaussian process, with  
zero at the origin, such that, for s; t >= 0:

$$\mathbb{E}[B_H(t)B_H(s)] = \frac{\sigma^2}{2}(|t|^{2H} + |s|^{2H} - |t-s|^{2H})$$

From this covariance, we get the self-similarity property:

$$\mathbb{E}[B_H(t)B_H(s)] = \sigma^2|t-s|^{kH}$$

This property states that the k-order absolute moment of the increments of duration  
|t-s| is proportional to |t-s|^(kH). Comparing two scales thus makes it possible to estimate H.  
If we focus on the two smallest scales, we indeed get the following estimator, for a time series of  
log-prices  

$$H_k = \frac{1}{k}log_2(\frac{(t-1)\Sigma_{i=1}^{t-2}|X_{i+2}-X_i|^k}{(t-2)\Sigma_{i=1}^{t-1}|X_{i+1}-X_i|^k})$$

### VIX

The VIX is a proxy for the implied volatility, computed considering
Standard & Poor’s 500 Index data.  
It widely regarded as a measure of turbulence in U.S. and global financial markets.  

By modeling the S&P 500 Index as a multifractional Brownian motion, we  
exploit the relationship between its Hurst exponent and the volatility to  
predict the VIX by a Distributed Lag model.

Assuming that the S&P 500 Index follows  
a multifractional Brownian motion, we exploit the theoretical relationship  
between its Hurst exponent and the volatility  

### Import Datas from Yahoo Finance

In [37]:
SP500 = yf.download('^GSPC',start='1980-09-01', end='2022-08-29', interval='1mo')

[*********************100%***********************]  1 of 1 completed


### Estimation

In [47]:
def estimation_h(omega,data):
    liste_h = []
    M1_t=0
    M2_t=0
    
    T_debut = 72
    
    # initialisation à t = 3 
    for i in range(1,T_debut+1):
        M1_t = M1_t + (omega**(T_debut-1-i))*((data[i+1]-data[i])**2)
        M2_t = M2_t+ (omega**(T_debut-2-i))*((data[i+2]-data[i])**2)
    
    H = (1/2)*np.log2( ((T_debut-1)*M2_t )/((T_debut-2)*M1_t ))
    
    liste_h.append(H)
    
    # récurence 
    
    for k in range (T_debut+1,len(data)-1):
        
        # calcul des moments suivants
        
        next_M1_t = omega*M1_t + (data[k+1]-data[k])**2
        next_M2_t = omega*M2_t + (data[k+1]-data[k-1])**2
        
        next_H = (1/2)*np.log2(  ((k-1)*next_M2_t) / ((k-2)*next_M1_t) )
        
        # ajoutdu nouveau H à notre liste de valeurs
        liste_h.append(next_H)
        
        M1_t = next_M1_t
        M2_t = next_M2_t
        
    
    return liste_h
    

In [50]:
hurst_sp500 = estimation_h(0.95, SP500)

[0.6868433585564595,
 0.6886288082210233,
 0.7237116940057272,
 0.7264013298784119,
 0.7122330856023055,
 0.6744889980219345,
 0.6436849335836405,
 0.664326608984817,
 0.6577840655144819,
 0.6555847036242467,
 0.6263959590327425,
 0.4694667726166025,
 0.5208831851215254,
 0.5202078339015407,
 0.5132580783979144,
 0.5002231338456121,
 0.5075272798392481,
 0.5042669332322881,
 0.47977410755374006,
 0.47004176928142527,
 0.470769645316781,
 0.4720151250132241,
 0.46389487761396936,
 0.48112997328778445,
 0.48355790909064533,
 0.48474889700857154,
 0.4870762750008053,
 0.4692066133269075,
 0.4546432800008051,
 0.46321990887101877,
 0.46260659473468096,
 0.43967256044866543,
 0.4465435133695733,
 0.4353504009855741,
 0.4303544532134543,
 0.42680785903967083,
 0.42517793417807626,
 0.3898213141331104,
 0.43433563138100545,
 0.454458006670633,
 0.4604461722497821,
 0.43597302602151655,
 0.3980089590492542,
 0.4462105802822452,
 0.41911151927299534,
 0.40345063709085216,
 0.3482201538461633,
 

###  The hurst exponent we have calculated above has been added to our csv data